In [2]:
import requests

In [3]:
import pandas as pd

/home/wolfram/miniconda2/lib/python2.7/site-packages/pandas/_libs/__init__.py:3: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .tslib import iNaT, NaT, Timestamp, Timedelta, OutOfBoundsDatetime
/home/wolfram/miniconda2/lib/python2.7/site-packages/pandas/__init__.py:26: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import (hashtable as _hashtable,
/home/wolfram/miniconda2/lib/python2.7/site-packages/pandas/core/dtypes/common.py:6: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import algos, lib
/home/wolfram/miniconda2/lib/python2.7/site-packages/pandas/core/util/hashing.py:7: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import hashing, tslib
/home/wolfram/miniconda2/lib/python2.7/site-packages/pandas/core/indexe

In [4]:
bbb_df = pd.read_csv('data/Ball_By_Ball.csv')

In [5]:
import sys
sys.version

'2.7.16 |Anaconda custom (64-bit)| (default, Mar 14 2019, 21:00:58) \n[GCC 7.3.0]'

In [6]:
player_df = pd.read_csv('data/Player.csv')

In [7]:
match_df = pd.read_csv('data/Match.csv')

In [8]:
playermatch_df = pd.read_csv('data/Player_match.csv')

In [9]:
team_df = pd.read_csv('data/Team.csv')

In [10]:
player_df.head(5)

,PLAYER_SK,Player_Id,Player_Name,DOB,Batting_hand,Bowling_skill,Country_Name
0,0,1,SC Ganguly,7/8/1972,Left-hand bat,Right-arm medium,India
1,1,2,BB McCullum,9/27/1981,Right-hand bat,Right-arm medium,New Zealand
2,2,3,RT Ponting,12/19/1974,Right-hand bat,Right-arm medium,Australia
3,3,4,DJ Hussey,7/15/1977,Right-hand bat,Right-arm offbreak,Australia
4,4,5,Mohammad Hafeez,10/17/1980,Right-hand bat,Right-arm offbreak,Pakistan


In [11]:
bbb_df.head(5)

,MatcH_id,Over_id,Ball_id,Innings_No,Team_Batting,Team_Bowling,Striker_Batting_Position,Extra_Type,Runs_Scored,Extra_runs,...,Fielder_match_SK,Fielder_SK,Bowler_match_SK,BOWLER_SK,PlayerOut_match_SK,BattingTeam_SK,BowlingTeam_SK,Keeper_Catch,Player_out_sk,MatchDateSK
0,598028,15,6,1,5,2,6.0,No Extras,4,0,...,-1,-1,20343,82,-1,4,1,0,0,20130420
1,598028,14,1,1,5,2,5.0,No Extras,1,0,...,-1,-1,20348,345,-1,4,1,0,0,20130420
2,598028,14,2,1,5,2,3.0,No Extras,1,0,...,-1,-1,20348,345,-1,4,1,0,0,20130420
3,598028,14,3,1,5,2,5.0,No Extras,1,0,...,-1,-1,20348,345,-1,4,1,0,0,20130420
4,598028,14,4,1,5,2,3.0,No Extras,0,0,...,-1,-1,20348,345,-1,4,1,0,0,20130420


In [32]:
team_name_id_map = dict(team_df.loc[:, ['Team_Name', 'Team_Id', ]].values)

In [38]:
bbb_df.Team_Batting = bbb_df.Team_Batting.replace(team_name_id_map).astype(int)
bbb_df.Team_Bowling = bbb_df.Team_Bowling.replace(team_name_id_map).astype(int)

In [39]:
def sixes(x):
    return sum(x == 6)
def fours(x):
    return sum(x == 4)
def run_rate(x):
    return (float(sum(x))/len(x))*100
def total_runs(x):
    return sum(x)

In [40]:
batting = bbb_df.groupby(['MatcH_id', 'Striker', 'Team_Bowling', 'Team_Batting']).agg({'Runs_Scored': [total_runs, sixes, fours, run_rate],
                                            }).reset_index()

batting.columns = batting.columns.map('{0[0]}|{0[1]}'.format) 
batting= batting.rename(columns={'MatcH_id|': 'match_id', 'Striker|': 'player_id', 'Team_Batting|': 'team_batting_id', 'Team_Bowling|': 'team_bowling_id',
                       'Runs_Scored|total_runs': 'total_runs', 'Runs_Scored|sixes': 'sixes', 'Runs_Scored|fours': 'fours', 'Runs_Scored|run_rate': 'run_rate', 
                                'Team_Bowling|': 'team_id_against', 'Team_Batting|': 'team_id'})
batting.head(5)

,match_id,player_id,team_id_against,team_id,total_runs,sixes,fours,run_rate
0,335987,1,2,1,10,0,2,83.333333
1,335987,2,2,1,158,13,10,205.194805
2,335987,3,2,1,20,1,1,100.000000
3,335987,4,2,1,12,0,1,100.000000
4,335987,5,2,1,5,0,1,166.666667


In [41]:
batting.dtypes

match_id             int64
player_id            int64
team_id_against      int64
team_id              int64
total_runs           int64
sixes                int64
fours                int64
run_rate           float64
dtype: object

In [42]:
batting_test  = batting[batting.match_id.astype(str).str.startswith('10')]
batting_train = batting[~batting.match_id.astype(str).str.startswith('10')]

In [43]:
import torch
from torch.utils import data

class IPLBattingDataset(data.Dataset):
    def __init__(self, player_id, team_id, team_id_against, total_runs):
        self.player_id = player_id
        self.team_id = team_id
        self.team_id_against = team_id_against
        self.total_runs = total_runs
        
    def __len__(self):
        return len(self.player_id)

    def __getitem__(self, index):
       
        return (self.player_id[index], self.team_id[index], self.team_id_against[index]), self.total_runs[index]
    


In [44]:
train_data = data.DataLoader(batting_train, batch_size=16)
test_data = data.DataLoader(batting_test, batch_size=16)


In [45]:
batting_test.shape
batting_train.shape

(8640, 8)

### batting model

In [47]:
n_players = batting.player_id.nunique()
n_teams = batting.team_id.nunique()
# n_teamsa

In [55]:
import torch
from torch import nn

In [56]:
class IPLNet(nn.Module):
    def __init__(self, n_players, n_teams, player_n_dim=16, team_n_dim=16):
        super(IPLNet, self).__init__()
        self.player_embeddings = nn.Embedding(num_embeddings=n_players, embedding_dim=player_n_dim)
#         print(n_teams)
        self.team_embeddings = nn.Embedding(num_embeddings=n_teams, embedding_dim=team_n_dim)
        
        n_features = player_n_dim + team_n_dim * 2
        self.linear_1 = nn.Linear(n_features, n_features // 2)
        self.linear_2 = nn.Linear(n_features // 2, n_features // 4)
        self.linear_3  = nn.Linear(n_features//4, 1)
        
    def forward(self, player_id, team_id, team_id_against):
        
        player_x = self.player_embeddings(player_id)
        team_x = self.team_embeddings(team_id)
        team_against_x = self.team_embeddings(team_id_against)
        
        x = torch.cat([player_x, team_x, team_against_x], dim=0)
        x = nn.Tanh(linear_1(x))
        x = nn.Tanh(linear_2(x))
        return torch.relu(x)

iplnet = IPLNet(n_players=n_players, n_teams=n_teams)
iplnet.cuda()

RuntimeError: cuda runtime error (35) : CUDA driver version is insufficient for CUDA runtime version at /opt/conda/conda-bld/pytorch_1533672544752/work/aten/src/THC/THCGeneral.cpp:74

In [53]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(iplnet.parameters())

In [54]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
for i in range(100):
    ytrain_pred = []
    ytrain_true = []
    for batch, (X, y) in enumerate(train_data):
        optimizer.zero_grad()
        y_pred = iplent(X)
        loss = loss_fn(y_pred, y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        ytrain_pred += list(y_pred.numpy())
        ytrain_true += list(y.numpy())
        print('\r', 'batch %d loss: %f' % (batch, running_loss/(batch+1)), end="")
        
    ytest_pred = []
    ytest_true = []
    with torch.no_grad():
        for X, y in test_data:
            y_pred = iplnet(X)
            loss = loss_fn(y_pred, y)
            running_loss += loss.item()
            ytest_pred += list(y_pred.numpy())
            ytest_true += list(y.numpy())
    
    print('-----------')
    print('epoch ', i)
    print('train mae: ', mean_absolute_error(ytrain_true, ytrain_pred))
    print('test mae: ', mean_absolute_error(ytest_true, ytest_pred))
            

In [137]:
batting.match_id.nunique()

636

In [19]:
batting.columns

Index([u'match_id', u'player_id', u'team_id_against', u'team_id',
       u'total_runs', u'sixes', u'fours', u'run_rate'],
      dtype='object')

In [14]:
import re

In [16]:
def outs(x):
    return sum(x != 'Not Applicable')
def economy_rate(x):
    return sum(x)
def maiden_over(x):
    return 1 if sum(x) == 0 else 0


In [17]:
import numpy as np

In [18]:

bowling_overwise = bbb_df.groupby(['MatcH_id', 'Bowler', 'Over_id', 'Team_Bowling', 'Team_Batting']).agg({'Out_type': outs, 'Runs_Scored': [maiden_over, economy_rate],
                                            }).reset_index()

bowling_overwise.columns = bowling_overwise.columns.map('{0[0]}|{0[1]}'.format) 
bowling = bowling_overwise.groupby(['MatcH_id|', 'Bowler|', 'Team_Bowling|', 'Team_Batting|']).agg({'Out_type|outs': sum, 'Runs_Scored|maiden_over': sum, 'Runs_Scored|economy_rate': np.mean}).reset_index()
bowling = bowling.rename(columns={'MatcH_id|': 'match_id',	'Bowler|': 'player_id',	'Out_type|outs': 'outs', 'Runs_Scored|maiden_over': 'maiden_overs',
                                  'Runs_Scored|economy_rate': 'economy_rate', 'Team_Bowling|': 'team_id', 'Team_Batting|': 'team_id_against'})
bowling.head(5)

,match_id,player_id,team_id,team_id_against,economy_rate,maiden_overs,outs
0,335987,1,1,2,5.00,0,3
1,335987,9,2,1,11.75,0,1
2,335987,10,2,1,22.00,0,0
3,335987,13,2,1,8.75,0,1
4,335987,14,2,1,9.25,1,0


### Batsman features

In [ ]:
'player', 'runs', '4s', '6s', 'strike_rate', 'opposition', 'ground', 'opposition_players_list'

### Bowler features

In [ ]:
'player', 'wickets', 'economy_rate', 'n_maiden_overs', 'opposition', 'ground', 'opposition_players_list'

In [ ]:
BATTING
=======
per run +0.5
per 4 +0.5
per 6 +1
50 +4
100 +8
0 -2
SR=60-70 -1
SR=50-60 -2
SR=<50 -3

BOWLING
=======
per wicket +10
4 wickets +4
5 wickets +4
maiden over +4
economy rate (runs/overs) <4 +3
economy rate (runs/overs) 4-5 +2
economy rate (runs/overs) 5-6 +1
economy rate (runs/overs) 9-10 -1
economy rate (runs/overs) 10-11 -2
economy rate (runs/overs) >11 -3

FIELDING
========
catch +4
stump/run out +6
runout thrower +4
runout catcher +2